<h1> <center> Assignment 06: Dynamic Indexing Algorithm </center>  </h1>

#### Following Python code is generated from one of the GenAI tool.

In [2]:
import os
from collections import defaultdict

class IndexSegment:
    def __init__(self):
        self.index = defaultdict(list)

    def add_document(self, doc_id, content):
        terms = content.lower().split()
        for position, term in enumerate(terms):
            self.index[term].append((doc_id, position))

    def merge(self, other):
        for term, postings in other.index.items():
            self.index[term].extend(postings)
        return self


class DynamicIndex:
    def __init__(self, merge_threshold=2, buffer_size=5):
        self.segments = []
        self.merge_threshold = merge_threshold
        self.buffer_size = buffer_size
        self.buffer = IndexSegment()
        self.buffer_count = 0

    def add_document(self, doc_id, content):
        self.buffer.add_document(doc_id, content)
        self.buffer_count += 1

        if self.buffer_count >= self.buffer_size:
            self._flush_buffer()

    def _flush_buffer(self):
        if self.buffer_count > 0:
            self.segments.append(self.buffer)
            self.buffer = IndexSegment()
            self.buffer_count = 0
            self._merge_segments()

    def _merge_segments(self):
        i = 0
        while i < len(self.segments) - 1:
            if len(self.segments) - i >= self.merge_threshold:
                merged = self.segments[i]
                for j in range(1, self.merge_threshold):
                    merged = merged.merge(self.segments[i + j])
                self.segments[i] = merged
                del self.segments[i + 1 : i + self.merge_threshold]
            i += 1

    def search(self, query):
        results = []
        for segment in self.segments:
            results.extend(segment.index.get(query.lower(), []))

        results.extend(self.buffer.index.get(query.lower(), []))
        return results

    def update_document(self, doc_id, new_content):

        for segment in self.segments:
            segment.index = defaultdict(list, {term: [(d, p) for d, p in postings if d != doc_id]
                                               for term, postings in segment.index.items()})
        self.buffer.index = defaultdict(list, {term: [(d, p) for d, p in postings if d != doc_id]
                                               for term, postings in self.buffer.index.items()})
        self.add_document(doc_id, new_content)

### 1. Go through the code. Write the Driver Code to execute the Dynamic Indexing.
[Try with dummy Data without reading files from secondary storage]

### 2. Write Explaination for each Method in both the classes.

### 3. Explain merge_threshold and buffer_size in the context on this code.

### 4. Next, execute the code on the 1000 documents dataset. https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

### 5. Write the driver code to execute the search document functionality. [Execute 2-3 examples]



### **1.** **Driver** Code

In [7]:
if __name__ == "__main__":
  index = DynamicIndex(merge_threshold=2, buffer_size=5)
  documents = {
    1: "Information retrieval is the process of obtaining information system resources that are relevant to an information need from a collection of those resources.",
    2: "Python is a versatile programming language used for web development, data analysis, artificial intelligence, and more.",
    3: "In information retrieval, indexing is crucial for efficient search and retrieval of data.",
    4: "Python libraries such as NumPy and Pandas are widely used for data manipulation and analysis.",
    5: "Information retrieval systems use algorithms to rank documents based on their relevance to the query.",
    6: "The Python programming language supports various libraries for machine learning, such as Scikit-learn and TensorFlow.",
    7: "Effective indexing techniques improve the performance of search engines by optimizing query processing.",
    8: "Python’s simplicity and readability make it a popular choice for beginners and professionals alike.",
    9: "Information retrieval systems often utilize data structures like inverted indexes to speed up search operations.",
    10: "Python's rich ecosystem includes libraries for web scraping, like BeautifulSoup and Scrapy, which are useful for gathering data for information retrieval."
  }

  # Add the documents to the index
  for doc_id,content in documents.items():
    index.add_document(doc_id,content)

  # Perform search queries
  print("Search results for 'information':", index.search("information"))
  print("Search results for 'indexing':", index.search("indexing"))
  print("Search results for 'documents':", index.search("documents"))
  print("Search results for 'threshold':", index.search("threshold"))
  print("Search results for 'python':", index.search("Python"))


  # Update document
  index.update_document(2, "Dynamic indexing can handle document updates seamlessly.")
  print("After update, search results for 'indexing':", index.search("indexing"))

Search results for 'information': [(1, 0), (1, 7), (1, 15), (3, 1), (5, 0), (9, 0), (10, 19)]
Search results for 'indexing': [(3, 3), (7, 1)]
Search results for 'documents': [(5, 7)]
Search results for 'threshold': []
Search results for 'python': [(2, 0), (4, 0), (6, 1)]
After update, search results for 'indexing': [(3, 3), (7, 1), (2, 1)]


### 2. Explaination for each Method in both the classes

#### **Class `IndexSegment`:**

- **`__init__(self):`**  
  Initializes an empty index using a `defaultdict` where each term points to a list of tuples, with each tuple containing the document ID and the position of the term in that document.

- **`add_document(self, doc_id, content):`**  
  Adds a document to the index. It tokenizes the content into terms (by splitting on spaces) and adds each term, along with its position in the document, to the index.

- **`merge(self, other):`**  
  Merges another `IndexSegment` into the current one by appending the posting lists of each term from `other` to the current index. This helps reduce the number of segments.

#### **Class `DynamicIndex`:**

- **`__init__(self, merge_threshold=2, buffer_size=5):`**  
  Initializes a dynamic index with a list of segments, a buffer segment, and a buffer size. The `merge_threshold` defines how many segments to merge at a time, and `buffer_size` defines how many documents can be stored in the buffer before it's flushed.

- **`add_document(self, doc_id, content):`**  
  Adds a document to the buffer. If the buffer reaches its size limit (as defined by `buffer_size`), it flushes the buffer to create a new segment.

- **`_flush_buffer(self):`**  
  Moves the buffer into the segments and resets the buffer. It also triggers merging of the segments if needed.

- **`_merge_segments(self):`**  
  Merges segments in the list of segments. It checks if the number of segments is greater than or equal to the `merge_threshold` and merges them together to keep the number of segments manageable.

- **`search(self, query):`**  
  Searches for a query term across all segments and the buffer. It returns a list of positions where the term is found.

- **`update_document(self, doc_id, new_content):`**  
  Removes all references to a document (by its `doc_id`) from both the segments and the buffer. After that, it re-adds the document with its updated content.


###  3. Explanation of `merge_threshold` and `buffer_size` in the Context of the Code

#### **buffer_size:**

- **Context in Code:**
  In the `DynamicIndex` class, `buffer_size` defines the maximum number of documents that can be added to the in-memory buffer before the buffer is "flushed." When the buffer reaches the specified size, it is moved to the list of segments, and a new buffer is created for further document additions.

- **Impact in the Code:**
  - **Document Handling:** A larger `buffer_size` allows more documents to be accumulated in memory before flushing. This means that fewer flush operations will occur, leading to the creation of fewer segments. This can be advantageous if you want to minimize the number of segments but need to handle larger memory usage.
  - **Memory Usage:** Larger buffers consume more memory as they hold more documents. This can be a trade-off between memory efficiency and the overhead of frequent buffer flushing.
  - **Performance:** By reducing the number of flushes, you may reduce the overhead of segment creation and improve indexing performance. However, if the buffer size is too large, it might lead to increased memory consumption, which could affect the system's overall performance.

#### **merge_threshold:**

- **Context in Code:**
  In the `DynamicIndex` class, `merge_threshold` specifies how many segments need to be present before a merge operation is triggered. When the number of segments meets or exceeds this threshold, the `merge_segments` method merges the segments to reduce the total number of segments in the index.

- **Impact in the Code:**
  - **Segment Management:** A lower `merge_threshold` results in more frequent merge operations. This means segments are merged together more often, which helps keep the number of segments smaller and more manageable. It ensures that the index does not grow indefinitely and remains efficient for searching.
  - **Processing Overhead:** Frequent merging can increase the processing cost, as each merge operation involves combining segments and updating the index. This might affect the system's performance if merging is computationally intensive.
  - **Search Efficiency:** By merging segments more frequently, you can improve search efficiency by reducing the number of segments that need to be searched. Fewer, larger segments can lead to faster search times compared to a larger number of smaller segments.



### 4. execute the code on the 1000 documents dataset

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
dataset = "/content/drive/MyDrive/CSE 419/Lab Assign-4/1000_documents"

In [13]:
index = DynamicIndex(merge_threshold=2, buffer_size=5)

    # Read and index each .txt file
for filename in os.listdir(dataset):
    if filename.endswith('.txt'):
        doc_id = filename  # Use filename as a unique identifier
        file_path = os.path.join(dataset, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            index.add_document(doc_id, content)

In [17]:
index.search("thinking")

[('entertainment_67.txt', 361),
 ('food_15.txt', 91),
 ('food_70.txt', 349),
 ('graphics_15.txt', 43),
 ('graphics_40.txt', 336),
 ('graphics_35.txt', 315),
 ('graphics_52.txt', 1154),
 ('graphics_52.txt', 1491),
 ('historical_90.txt', 662),
 ('historical_90.txt', 756),
 ('medical_308.txt', 13),
 ('medical_41.txt', 57),
 ('politics_229.txt', 444),
 ('sport_40.txt', 381),
 ('sport_65.txt', 228),
 ('sport_93.txt', 99),
 ('sport_93.txt', 127),
 ('technologie_56.txt', 123),
 ('technologie_50.txt', 305),
 ('technologie_78.txt', 508)]

In [18]:
while True:
  query = input("Enter your search query: ").strip()

  if query.lower() == 'n':
    print("Stopping the search.")
    break

  results = index.search(query)
  print(f"\nSearch results for query '{query}':")
  if results:
    for doc_id, position in results:
      print(f"Document ID: {doc_id}, Position: {position}")
  else:
    print("No results found.")
  print("\n")

Enter your search query: hello

Search results for query 'hello':
Document ID: graphics_10.txt, Position: 0


Enter your search query: good

Search results for query 'good':
Document ID: business_11.txt, Position: 164
Document ID: business_30.txt, Position: 291
Document ID: business_30.txt, Position: 460
Document ID: business_4.txt, Position: 136
Document ID: business_27.txt, Position: 131
Document ID: business_39.txt, Position: 196
Document ID: business_37.txt, Position: 176
Document ID: business_34.txt, Position: 124
Document ID: business_21.txt, Position: 19
Document ID: business_40.txt, Position: 64
Document ID: business_50.txt, Position: 334
Document ID: business_69.txt, Position: 335
Document ID: business_53.txt, Position: 166
Document ID: business_56.txt, Position: 168
Document ID: business_78.txt, Position: 299
Document ID: business_82.txt, Position: 166
Document ID: business_81.txt, Position: 263
Document ID: business_81.txt, Position: 418
Document ID: entertainment_25.txt, Po